# Standard Library

Here we have a collection of standard functions that help writing stringdale diagrams effectively

In [ ]:
# |default_exp utils

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

from stringdale.doc import show_doc
import pytest

In [ ]:
#| export
from stringdale.core import get_git_root, load_env, checkLogs,  json_render,json_undeclared_vars
from stringdale.mappings import map_object

import openai 
from pydantic import BaseModel, create_model
from typing import Optional, Dict, Any, List, Union
import json
import re
from parse import parse
from pathlib import Path
from enum import Enum
from pydantic import BaseModel
import logging
from joblib import Memory
import os


In [ ]:
#| export
load_env()
logger = logging.getLogger(__name__)

# Create a cache directory in the user's home directory
cache_dir = get_git_root() / '.cache' / 'image_text_cache'
cache_dir.mkdir(parents=True, exist_ok=True)
memory = Memory(location=str(cache_dir), verbose=0)

# Initialize the Memory object for caching
memory = Memory(cache_dir, verbose=0)


## Nested Conditions made simple

In [ ]:
# | export
import re

class _RegexCondition:
    def __init__(self, regex):
        self.pattern = regex
        self.regex = re.compile(regex)
    def __call__(self, x):
        if not isinstance(x, str):
            return False
        return self.regex.search(x) != None

    def __str__(self):
        return f"/{self.pattern}/"
    def __repr__(self):
        return str(self)
    



In [ ]:
#| hide
assert _RegexCondition(r"\d+")("123")
assert _RegexCondition(r"\d+")("123abc")
assert not _RegexCondition(r"\d+")("abs")
assert not _RegexCondition(r"\d+$")("123abc")
assert _RegexCondition(".*")("123")
assert _RegexCondition(".*")("1123abc23")

In [ ]:
#| export
from stringdale.base import _get_func_name


In [ ]:
#| export
class _FuncCondition:
    def __init__(self, func):
        self.func = func
    def __call__(self,*args,**kwargs):
        return self.func(*args,**kwargs)
    def __str__(self):
        return f"{_get_func_name(self.func)}"
    def __repr__(self):
        return str(self)


In [ ]:
#| export

# TODO make it so that we can do several or conditions and it will results in a single _ORCondition
class _ORCondition:
    def __init__(self, cond1, cond2):
        self.cond1 = cond1
        self.cond2 = cond2
        self.name = f"({self.cond1} | {self.cond2})"
    def __call__(self, x):
        return self.cond1(x) or self.cond2(x)

    def __and__(self, other):
        return _ANDCondition(self, other)
    def __or__(self, other):
        return _ORCondition(self, other)

    def __str__(self):
        return self.name
    def __repr__(self):
        return str(self)

class _ANDCondition:
    def __init__(self, cond1, cond2):
        self.cond1 = cond1
        self.cond2 = cond2
        self.name = f"({self.cond1} & {self.cond2})"
    def __call__(self, x):
        return self.cond1(x) and self.cond2(x)

    def __and__(self, other):
        return _ANDCondition(self, other)
    def __or__(self, other):
        return _ORCondition(self, other)

    def __str__(self):
        return self.name
    def __repr__(self):
        return str(self)



In [ ]:
#| export
from stringdale.mappings import map_object,parse_edge_descriptor,object_to_args_kwargs
from typing import Callable,Optional,Dict,Any,List,Union

In [ ]:
#| export
class Condition:
    """
        A utility function for creating condition functions using stringdale's port mapping logic.
        
        Args:
            func: A function or string to match against the input. If string, it will be interpreted as a regex pattern to match the input against.
            mapping: A mapping of the input to the function. by default, the input is the first argument of the function.
            name: A name for the condition. by default, is func_name->mapping_string
    """
    def __init__(self,func:Union[Callable,str],mapping:Optional[str]=None, name:Optional[str]=None):

        if isinstance(func, str):
            func = _RegexCondition(func)
        elif callable(func):
            func = _FuncCondition(func)
        else:
            raise ValueError(f"Last part of condition should be a string or callable, not {last_part}")
        
        out = parse_edge_descriptor("**=a",start='mapping_expr')

        self.map_string = mapping
        if mapping is None:
            mapping = {0:('.',)}
        else:
            mapping = parse_edge_descriptor(mapping,start='mapping_expr')
        self.mapping = mapping
        self.func = func
        self.name = name
                    
    def __call__(self, x):
        try:
            mapped_x = map_object(x,self.mapping)
        except Exception as e:
            return False
        args,kwargs = object_to_args_kwargs(mapped_x)
        return self.func(*args,**kwargs)
    
    def __and__(self, other):
        return _ANDCondition(self, other)
    def __or__(self, other):
        return _ORCondition(self, other)

    def __str__(self):
        if self.name is not None:
            return self.name
        elif self.map_string is None:
            return f"Condition( {self.func} )"
        else:
            return f"Condition( {self.map_string} ) -> ({self.func})"
    def __repr__(self):
        return str(self)

In [ ]:
def is_module_2(x):
    return x%2==0

is_input_even = Condition(is_module_2,'(0=input)',name='input is even')

assert is_input_even({"input": 2})
assert not is_input_even({"input": 3})

In [ ]:
def is_sum_42(x,y):
    return x+y==42

is_io_42 = Condition(is_sum_42,'(x=input,y=output)',name='i+o=42?')

assert is_io_42({"input": 2,'output':40})

In [ ]:
C = Condition

complex_cond = C("^3",'(0=input)',name='input is 3') & C("^5",'(0=output)',name='output is 5') | C("^2",'(0=input)')
complex_cond

((input is 3 & output is 5) | Condition( (0=input) ) -> (/^2/))

In [ ]:
assert complex_cond({"input": "345", "output": "567"})
assert complex_cond({"input": "244", "output": "567"})
assert not complex_cond({"input": "145", "soutput": "567"})
assert not complex_cond({"input": "345", "output": "67"})
assert not complex_cond({"not_input": "145"})
assert complex_cond({ "input": "244"})

In [ ]:
simple_cond = C("^2")
assert not simple_cond({"input": "245", "output": "567"})
assert not simple_cond({"input": "245"})
assert simple_cond("245")


## Structuring objects

In [ ]:
#| export
from stringdale.mappings import parse_edge_descriptor,set_access,access

In [ ]:
#| export
class StructureJson():
    """
        A class for restructuring JSON objects by nested paths
        This class allows restructuring JSON objects by specifying path assignments in the form
        'target_path=source_path'. It parses these assignments and creates a new JSON object
        with the specified structure.

        Args:
            assignments: Variable number of strings specifying path assignments in the format
                'target_path=source_path'

        Returns:
            A new JSON object with the specified structure.
    """
    def __init__(self,*assignments):
        self.path_strings = assignments
        self.assignment_paths = [parse_edge_descriptor(path,start='accessor_assignment') for path in assignments]
    def __call__(self,**kwargs):
        target_obj = {}
        for write_path,read_path in self.assignment_paths:
            sub_obj = access(kwargs,read_path,raw=True)
            set_access(target_obj,write_path,sub_obj)
        return target_obj
    def __str__(self):
        return f"StructureJson({', '.join(self.path_strings)})"
    def __repr__(self):
        return str(self)


In [ ]:
StructureJson('a.b.c=d.e.f','a.b.d=d.x')

StructureJson(a.b.c=d.e.f, a.b.d=d.x)

In [ ]:
source_obj = {'d':{'e':{'f':'foo'},'x':'bar'}}
out = StructureJson('a.b.c=father.d.e.f','a.b.d=father.d.x')(father=source_obj)
assert out == {'a':{'b':{'c':'foo','d':'bar'}}}


## JsonRenderer

In [ ]:
#| export
from stringdale.core import json_render,json_undeclared_vars

In [ ]:
#| export
class JsonRenderer():
    """
        A class for rendering JSON objects with nested jinja2 templates.

        Allows setting template variables both during init and when calling the object.

        Args:
            json_obj: A JSON object to render.
            **kwargs: Context variables to use in the rendering.

        Returns:
            A rendered JSON object.
    """
    def __init__(self,json_obj,**kwargs):
        self.json_obj = json_obj
        self.baked_kwargs = kwargs
    def __call__(self,**kwargs):
        full_kwargs = {**self.baked_kwargs,**kwargs}
        return json_render(self.json_obj,context=full_kwargs)

    def _missing_keys(self):
        template_vars = json_undeclared_vars(self.json_obj)
        return template_vars - set(self.baked_kwargs.keys())

    def __str__(self):
        return f'JsonRenderer(missing_keys={self._missing_keys()})'

    def __repr__(self):
        return str(self)


In [ ]:
template_json = {
    'name': '{{name}}',
    'age': '{{age}}',
    'city_path': '{%for city in cities%} {{city}} - {%endfor%}'
}

rend = JsonRenderer(template_json,age=30,cities=['SF','LA','NY'])
rend


JsonRenderer(missing_keys={'name'})

In [ ]:
out = rend(name='Dean')
assert out == {'name': 'Dean', 'age': '30', 'city_path': 'SF -  LA -  NY -'},out


## Export

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()